In [1]:
!pip install autokeras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.6/148.6 KB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.3/167.3 KB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 466.8/466.8 KB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 65.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 48.3 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
import numpy as np
import autokeras as ak

## Data Format
The AutoKeras ImageRegressor is quite flexible for the data format.

For the image, it accepts data formats both with and without the channel
dimension. The images in the MNIST dataset do not have the channel dimension.
Each image is a matrix with shape (28, 28). AutoKeras also accepts images of
three dimensions with the channel dimension at last, e.g., (32, 32, 3), (28,
28, 1).

For the regression targets, it should be a vector of numerical values.
AutoKeras accepts numpy.ndarray.

We also support using tf.data.Dataset format for the training data. In this
case, the images would have to be 3-dimentional.


In [3]:

(x_train, y_train), (x_test, y_test) = mnist.load_data()
print("##########################")
print("Shape matrix train:")
print(x_train.shape)  # (60000, 28, 28, 1)
print(y_train.shape)  # (60000, 10)
print("Shape matrix test:")
print(x_test.shape)  # (60000, 28, 28, 1)
print(y_test.shape)  # (60000, 10)

# Reshape the images to have the channel dimension.
x_train = x_train.reshape(x_train.shape + (1,))
x_test = x_test.reshape(x_test.shape + (1,))
y_train = y_train.reshape(y_train.shape + (1,))
y_test = y_test.reshape(y_test.shape + (1,))

print("##########################")
print("Shape tensor train:")
print(x_train.shape)  # (60000, 28, 28, 1)
print(y_train.shape)  # (60000, 10)
print("Shape tensor test:")
print(x_test.shape)  # (10000, 28, 28, 1)
print(y_test.shape)  # (10000, 10)


TRAIN_BUF=6000
BATCH_SIZE=64
train_set = tf.data.Dataset.from_tensor_slices(((x_train,), (y_train,))).batch(BATCH_SIZE, drop_remainder=False)
test_set = tf.data.Dataset.from_tensor_slices(((x_test,), (y_test,))).batch(BATCH_SIZE, drop_remainder=False)
# train_set = tf.data.Dataset.from_tensor_slices(((x_train,), (y_train,))).shuffle(TRAIN_BUF).batch(BATCH_SIZE, drop_remainder=True)
# test_set = tf.data.Dataset.from_tensor_slices(((x_test,), (y_test,))).shuffle(TRAIN_BUF).batch(BATCH_SIZE, drop_remainder=True)
# train_setBZ = train_set.batch(32)
# test_setBZ = test_set.batch(32)
print("##########################")
print("Type tensor train:")
print(type(train_set))
print("Type tensor test:")
print(type(test_set))



11490434/11490434 [==============================] - 0s 0us/step
##########################
Shape matrix train:
(60000, 28, 28)
(60000,)
Shape matrix test:
(10000, 28, 28)
(10000,)
##########################
Shape tensor train:
(60000, 28, 28, 1)
(60000, 1)
Shape tensor test:
(10000, 28, 28, 1)
(10000, 1)
##########################
Type tensor train:
<class 'tensorflow.python.data.ops.dataset_ops.BatchDataset'>
Type tensor test:
<class 'tensorflow.python.data.ops.dataset_ops.BatchDataset'>


In [4]:
# coonvert obs datasets (train and test) to numpy array again
def OPSdatasetbatchToNP(train_set, test_set):
  train_ds = train_set.unbatch()
  images_train = np.concatenate(list(train_ds.map(lambda x, y: x)))
  labels_train = np.concatenate(list(train_ds.map(lambda x, y: y)))

  test_ds = test_set.unbatch()
  images_test = np.concatenate(list(test_ds.map(lambda x, y: x)))
  labels_test = np.concatenate(list(test_ds.map(lambda x, y: y)))
  
  return images_train, labels_train, images_test, labels_test

images_trainNP, labels_trainNP, images_testNP, labels_testNP = OPSdatasetbatchToNP(train_set, test_set)

print(images_trainNP.shape)
print(labels_trainNP.shape)
print(images_testNP.shape)
print(labels_testNP.shape)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


(60000, 28, 28, 1)
(60000, 1)
(10000, 28, 28, 1)
(10000, 1)


In [10]:
def AkModel(max_trials=1):  
  AKmodel = ak.ImageRegressor(overwrite=True, max_trials=max_trials, distribution_strategy=tf.distribute.MirroredStrategy())
  return AKmodel


In [11]:
AKmodel = AkModel(max_trials=1)
# Feed the tensorflow Dataset to the model train.
AKmodel.fit(train_set, epochs=2, shuffle=True)
# Predict with the best model.
predicted_y = AKmodel.predict(test_set)
# Evaluate the best model with testing data.
print(AKmodel.evaluate(test_set))

Trial 1 Complete [00h 03m 12s]
val_loss: 0.7550757527351379

Best val_loss So Far: 0.7550757527351379
Total elapsed time: 00h 03m 12s
Epoch 1/2
938/938 [==============================] - 88s 53ms/step - loss: 2.9261 - mean_squared_error: 2.9261
Epoch 2/2
938/938 [==============================] - 49s 52ms/step - loss: 0.8708 - mean_squared_error: 0.8708


157/157 [==============================] - 5s 20ms/step - loss: 2.4439 - mean_squared_error: 2.4439
[2.443902015686035, 2.443902015686035]


In [12]:
AKmodel = AkModel(max_trials=1)
# Feed the numpy to the model train.
AKmodel.fit(images_trainNP, labels_trainNP, epochs=2, shuffle=True)
# Predict with the best model.
predicted_y = AKmodel.predict(images_testNP)
# Evaluate the best model with testing data.
print(AKmodel.evaluate(images_testNP,labels_testNP))

Trial 1 Complete [00h 03m 44s]
val_loss: 0.5378710627555847

Best val_loss So Far: 0.5378710627555847
Total elapsed time: 00h 03m 44s
Epoch 1/2
1875/1875 [==============================] - 129s 48ms/step - loss: 2.5927 - mean_squared_error: 2.5927
Epoch 2/2
1875/1875 [==============================] - 86s 46ms/step - loss: 0.8639 - mean_squared_error: 0.8639


313/313 [==============================] - 7s 14ms/step - loss: 2.3692 - mean_squared_error: 2.3692
[2.369199514389038, 2.369199514389038]


## Reference
[ImageRegressor](/image_regressor),
[AutoModel](/auto_model/#automodel-class),

